# 01 - Descarga y Limpieza del Dataset de Steam Reviews

**Materia:** Redes Neuronales Profundas — UTN FRM

**Objetivo:** Descargar el dataset de reseñas de Steam desde Kaggle, realizar una limpieza exhaustiva del texto y preparar un CSV balanceado para las etapas posteriores de tokenización y entrenamiento.

---

## Descripción del Dataset

Utilizamos el dataset [Steam Reviews](https://www.kaggle.com/datasets/andrewmvd/steam-reviews) de Kaggle (autor: andrewmvd). Este dataset contiene reseñas reales de usuarios de la plataforma Steam.

**Columnas relevantes:**
- `review_text`: texto libre de la reseña del usuario
- `review_score`: sentimiento de la reseña (1 = positivo, -1 = negativo)

**Tarea:** Clasificación binaria de sentimiento (positivo vs negativo).

## ¿Por qué limpiar el texto?

Las reseñas de Steam contienen gran cantidad de "ruido" que puede confundir al modelo:
- Arte ASCII (dibujos con caracteres especiales)
- URLs y enlaces
- Emotes con asteriscos (\*joins server\*)
- Puntuación excesiva (!!!!!!)
- Caracteres especiales y Unicode

Una limpieza agresiva del texto permite que BERT se enfoque en las palabras reales que expresan sentimiento.

## 1. Importación de Librerías

In [ ]:
import os
import re
import pandas as pd
import numpy as np

## 2. Configuración de Parámetros

- **SAMPLE_SIZE = 50,000:** Tomamos 25,000 reseñas positivas y 25,000 negativas para tener un dataset balanceado.
- **MIN_WORD_COUNT = 5:** Descartamos reseñas con menos de 5 palabras (no aportan información útil).
- **MAX_WORD_COUNT = 200:** Limitamos la longitud para evitar reseñas extremadamente largas que podrían ser spam.

In [ ]:
RAW_DATA_PATH = "../data/dataset.csv"        # CSV descargado de Kaggle
CLEAN_DATA_PATH = "../data/clean_reviews.csv"  # CSV limpio de salida
SAMPLE_SIZE = 50000                             # 25K positivas + 25K negativas
MIN_WORD_COUNT = 5                              # Mínimo de palabras por reseña
MAX_WORD_COUNT = 200                            # Máximo de palabras por reseña
RANDOM_SEED = 42

## 3. Carga del Dataset Crudo

El dataset de Kaggle puede venir en uno o más archivos CSV. Esta función maneja ambos casos.

In [ ]:
def load_raw_data(path):
    """Carga el CSV crudo del dataset de Kaggle."""
    if os.path.isfile(path):
        print(f"Cargando datos desde {path}...")
        df = pd.read_csv(path)
    else:
        data_dir = os.path.dirname(path)
        csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv') and 'clean' not in f]
        if not csv_files:
            raise FileNotFoundError(
                f"No se encontraron archivos CSV en {data_dir}.\n"
                "Descargá el dataset con: kaggle datasets download -d andrewmvd/steam-reviews"
            )
        print(f"Archivos encontrados: {csv_files}")
        dfs = [pd.read_csv(os.path.join(data_dir, f)) for f in csv_files]
        df = pd.concat(dfs, ignore_index=True)

    print(f"Datos cargados: {df.shape[0]:,} filas, {df.shape[1]} columnas")
    print(f"Columnas: {list(df.columns)}")
    return df

In [ ]:
df = load_raw_data(RAW_DATA_PATH)

### Exploración Inicial del Dataset

In [ ]:
df.head()

In [ ]:
print(f"Forma: {df.shape}")
print(f"\nTipos de datos:\n{df.dtypes}")
print(f"\nValores nulos:\n{df.isnull().sum()}")

In [ ]:
print("Distribución de review_score:")
print(df['review_score'].value_counts())

## 4. Funciones de Limpieza de Texto

### 4.1 Función `clean_text`

Aplica una limpieza agresiva al texto de cada reseña:

1. **Eliminar URLs:** Los enlaces no aportan información de sentimiento.
2. **Eliminar arte ASCII:** Caracteres decorativos comunes en Steam (░▒▓█).
3. **Eliminar emotes con asteriscos:** Patrones como \*applauds\*.
4. **Normalizar puntuación:** Reducir repeticiones excesivas (!!!!! → !!).
5. **Eliminar caracteres especiales:** Mantener solo letras, espacios y puntuación básica.
6. **Convertir a minúsculas:** Necesario para `bert-base-uncased`.

In [ ]:
def clean_text(text):
    """Limpia agresivamente el texto de una reseña de Steam."""
    if not isinstance(text, str):
        return ""
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'[░▒▓█▄▀■□▪▫●○◆◇♠♣♥♦♪♫☆★►◄▲▼←→↑↓]+', '', text)
    text = re.sub(r'\*[^*]+\*', '', text)
    text = re.sub(r'[•►▪▸‣⁃]', '', text)
    text = re.sub(r'(\d+)\s*/\s*(\d+)', r'\1 out of \2', text)
    text = re.sub(r'\d{3,4}\s*x\s*\d{3,4}', '', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'([!?.]){3,}', r'\1\1', text)
    text = re.sub(r"[^a-zA-Z\s.,!?'-]", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip().lower()
    return text

### 4.2 Función de Validación

Filtra reseñas que no son útiles para entrenamiento: muy cortas (< 5 palabras), muy largas (> 200 palabras), o con muy pocos caracteres únicos (spam).

In [ ]:
def is_valid_review(text, min_words=MIN_WORD_COUNT, max_words=MAX_WORD_COUNT):
    """Verifica si una reseña limpia es válida para entrenamiento."""
    if not isinstance(text, str) or len(text.strip()) == 0:
        return False
    words = text.split()
    if len(words) < min_words or len(words) > max_words:
        return False
    if len(set(text.replace(' ', ''))) < 5:
        return False
    return True

### Ejemplo de Limpieza

In [ ]:
ejemplo_raw = df.iloc[0]['review_text'] if 'review_text' in df.columns else df.iloc[0][df.columns[0]]
ejemplo_clean = clean_text(str(ejemplo_raw))
print("ORIGINAL:")
print(str(ejemplo_raw)[:300])
print("\nLIMPIO:")
print(ejemplo_clean[:300])

## 5. Procesamiento Completo y Balanceo de Clases

### ¿Por qué balancear clases?

Un dataset desbalanceado puede sesgar al modelo hacia la clase mayoritaria. Al tener exactamente 25,000 muestras de cada clase, el modelo aprende igualmente bien a detectar ambos sentimientos.

In [ ]:
def process_and_save(df, output_path, sample_size, seed):
    """Procesa el DataFrame: limpia, filtra, balancea y guarda."""
    print("--- Limpieza de texto ---")
    text_col, score_col = None, None
    for col in df.columns:
        col_lower = col.lower()
        if 'review_text' in col_lower: text_col = col
        if 'review_score' in col_lower: score_col = col
    if text_col is None: text_col = 'review_text'
    if score_col is None: score_col = 'review_score'
    print(f"Columna de texto: {text_col}, Columna de score: {score_col}")

    df = df.dropna(subset=[text_col])
    print(f"Filas después de eliminar nulos: {df.shape[0]:,}")

    print("Limpiando texto...")
    df['clean_text'] = df[text_col].apply(clean_text)
    df['is_valid'] = df['clean_text'].apply(is_valid_review)
    df = df[df['is_valid']].copy()
    print(f"Filas después de filtrar inválidas: {df.shape[0]:,}")

    if df[score_col].dtype == object:
        df['label'] = (df[score_col].str.lower() == 'recommended').astype(int)
    else:
        df['label'] = (df[score_col] > 0).astype(int)

    print(f"\nDistribución: Pos={df['label'].sum():,}, Neg={(df['label']==0).sum():,}")

    half = sample_size // 2
    pos = df[df['label'] == 1]
    neg = df[df['label'] == 0]
    n = min(half, len(pos), len(neg))
    print(f"Muestreando {n:,} de cada clase (total: {n*2:,})")

    balanced = pd.concat([
        pos.sample(n=n, random_state=seed),
        neg.sample(n=n, random_state=seed)
    ]).sample(frac=1, random_state=seed)

    result = balanced[['clean_text', 'label']].reset_index(drop=True)
    result.columns = ['text', 'label']
    result.to_csv(output_path, index=False)

    print(f"\nGuardado: {output_path}")
    print(f"Total: {len(result):,}, Distribución: {result['label'].value_counts().to_dict()}")
    return result

In [ ]:
result = process_and_save(df, CLEAN_DATA_PATH, SAMPLE_SIZE, RANDOM_SEED)

## 6. Verificación del Dataset Limpio

In [ ]:
for i, row in result.head(5).iterrows():
    label_str = "POSITIVA" if row['label'] == 1 else "NEGATIVA"
    print(f"[{label_str}] {row['text'][:150]}...\n")

In [ ]:
result['word_count'] = result['text'].apply(lambda x: len(x.split()))
print("Estadísticas de longitud (palabras):")
print(result['word_count'].describe())

## Resumen

1. Cargamos el dataset crudo de Steam Reviews de Kaggle.
2. Aplicamos limpieza agresiva del texto (URLs, arte ASCII, caracteres especiales, etc.).
3. Filtramos reseñas inválidas (muy cortas, muy largas, spam).
4. Balanceamos las clases (25,000 positivas + 25,000 negativas).
5. Guardamos en `data/clean_reviews.csv`.

**Siguiente paso:** Tokenizar las reseñas usando el tokenizer de BERT.